<a href="https://colab.research.google.com/github/RagozinaMarina/wanca/blob/master/wanca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Вывод по всему, что представлено ниже**

Добавление в качестве признаков целых слов немного улучшает работу модели. 

Добавление частотных слов не улучшает качество модели, а может даже понижать (например, при отборе значимых признаков и учете tf-idf)

## Обработка исходных файлов


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
FILES_DIR = '/content/drive/MyDrive/python_dz/wanca2016-src/txt'

In [ ]:
name_files = sorted(os.listdir(FILES_DIR))

In [ ]:
from collections import defaultdict

In [ ]:
texts = defaultdict(list)
for name in name_files:
  text_file = os.path.join(FILES_DIR, name)
  with open(text_file, "r", encoding="utf8") as fin:
    name_lang = re.sub('.txt', '', name)
    for line in fin:
      line = line.strip()
      text = line.split()
      sent = ''
      for i in range(len(text)-1): #последний элемент text - ссылка на текст, она нам не нужна
        sent += text[i]
        if i != len(text)-2:
          sent+=' '
      text = sent+'\t'+name_lang
      texts[name_lang].append(text)
print(texts.keys())

In [ ]:
for key in texts.keys():
  print(key, len(texts[key]))

In [ ]:
print(texts['vot'])

['A inehmized pyytäväd kaloi; pelkääväd, a yhskõikk pyytäväd.\tvot', 'A ize kuulimma, etti ken-leeb tulõb põõsai möö.\tvot', 'A karu, näb, eb tahtonnu meyed tarttua, vait tahtõ täm taitaameyed irmuttaa vähäkõizõõ.\tvot', 'A siält karu tuli vällää.\tvot', '”Ken kase võib õlla täällä?” – tõintõisiilt kysyzimmä.\tvot', 'Kiireessii teimmä, kalakaiccõa pelkäzimmä.\tvot', 'Kõikkilaill on täätävä, etti siäll, Kamčadgall, on palyo kalota, mõnõllaisia lõhia. Noh, rahvaz käyb, pyyväb kaloita, ked võivad.\tvot', 'Kõikk irmuzivad, viskazivad jalgad pihalõõ ja johsivad ken kuhõõ.\tvot', 'Menimmä kaukalõõ meccää, õikessi karunurkkaa.\tvot', 'Miä ize menin puhõõ, niku lintu lentäzin.\tvot', 'Muitõšši kalakaiccõ on paikall: võtab kiini, štrafib.\tvot', 'Mõni vootta tagaaz miä elin Kamčadgall.\tvot', 'Noh, mööki uzõin käyzimmä kalassamaa.\tvot', 'Noh, võtimma ja panimma kalad värcciisee.\tvot', 'Palvõlin siäl floottaza.\tvot', 'Tämä, karu, vaatahti meyee päälee, märizi ja ajõ meyed takaa vähäkõizõõ.\tv

In [ ]:
output_dir = '/content/drive/MyDrive/python_dz/wanca2016-src'

In [ ]:
outfile = os.path.join(output_dir, 'data.txt')
with open(outfile, "w", encoding="utf8") as fout:
  for key, values in texts.items():
    for sent in values:
      fout.write('{}\n'.format(sent))

Предварительно выключим предупреждения о сходимости

In [ ]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

## Чтение данных



In [ ]:
# читаем данные
def read_infile(infile):
    texts, labels = [], []
    with open(infile, "r", encoding="utf8") as fin:
        for line in fin:
            line = line.strip()
            if line == "":
                continue
            text, label = line.split("\t")
            texts.append(text)
            labels.append(label)
    return texts, labels

In [ ]:
data_dir = '/content/drive/MyDrive/python_dz/wanca2016-src/txt_processed/data.txt'

In [ ]:
data, data_labels = read_infile(data_dir)

In [ ]:
#разделим на train и test
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(
    data, data_labels, train_size=0.75, stratify=data_labels, random_state = 12)

## Преобразование данных в вектора




In [ ]:
from collections import defaultdict

# делим тексты на слова
def tokenize(text):
    words = text.split()
    words = [word.strip(",.\"()„«»;:") for word in words] # убираем знаки препинания
    words = [word for word in words if word != ""] # удаляем пустые слова
    return words

def get_ngrams_from_word(word, ngram_length=3, sub_word = False):
    answer = []
    for curr_ngram_length in range(1, min(ngram_length, len(word))+1):
        for end in range(curr_ngram_length, len(word)+1):
            start = end-curr_ngram_length
            answer.append(word[start:end])
    return answer

def get_ngrams_from_text(text, ngram_length=3, to_lower=True, whole_word = False):
    if to_lower:
        text = text.lower()
    ngrams = defaultdict(int)
    for word in tokenize(text):
        word_ngrams = get_ngrams_from_word(word, ngram_length=ngram_length)
        if whole_word is True and len(word) > ngram_length: #учитываем все слово целиком
          word_ngrams.append(word) 
        for ngram in word_ngrams:
            ngrams[ngram] += 1
    return ngrams


In [ ]:
import tqdm

class DataProcessor:

    def __init__(self, ngram_length=3, min_count=1, whole_word = False, sub_word = False):
        self.ngram_length = ngram_length
        self.min_count = min_count
        self.whole_word = whole_word
        self.sub_word = sub_word

    def fit(self, data):
        # первый проход: собираем счётчики энграмм
        ngram_counts = defaultdict(int)
        for text in tqdm.tqdm_notebook(data): # for text in data:
            text_ngram_counts = get_ngrams_from_text(text, ngram_length=self.ngram_length, whole_word = self.whole_word)
            for ngram in text_ngram_counts:
                ngram_counts[ngram] += 1
        # оставляем только частотные 
        if self.sub_word == True: #оставляем очень частотные слова
          self.ngrams = sorted(ngram for ngram, count in ngram_counts.items() if ((count >= self.min_count and len(ngram) <= self.ngram_length) or (len(ngram) > self.ngram_length and count >= 50)))
        else: #оставляем частотные
          self.ngrams = sorted(ngram for ngram, count in ngram_counts.items() if (count >= self.min_count))
        self.ngram_codes = {ngram: i for i, ngram in enumerate(self.ngrams)}
        print("{} энграмм в словаре.".format(len(self.ngrams)))
        return self

    def transform(self, data):
        return [self.transform_text(text) for text in data]

    def transform_text(self, text):
        ngrams = get_ngrams_from_text(text, ngram_length=self.ngram_length)
        answer = [0] * len(self.ngrams)
        for ngram, count in ngrams.items():
            code = self.ngram_codes.get(ngram)
            if code is not None:
                answer[code] = count
        return answer

In [ ]:
from scipy.sparse import csr_matrix

class SparseDataProcessor(DataProcessor):

    def transform(self, data):
        values, rows, columns = [], [], []
        for i, text in enumerate(tqdm.tqdm_notebook(data)):
            ngrams = get_ngrams_from_text(text, ngram_length=self.ngram_length, whole_word = self.whole_word)
            for ngram, count in ngrams.items():
                if len(ngram) > self.ngram_length and count < 50:
                  continue
                code = self.ngram_codes.get(ngram)
                if code is not None:
                    values.append(count)
                    rows.append(i)
                    columns.append(code)
        answer = csr_matrix((values, (rows, columns)), shape=(len(data), len(self.ngram_codes)))
        return answer

    def transform_text(self, text):
        raise NotImplementedError("Для разреженных данных не определено преобразование отдельного текста.")

## Преобразование данных и обучение модели

In [ ]:
data_processor = SparseDataProcessor(min_count=3, ngram_length=3, whole_word = True, sub_word = True)
data_processor.fit(train_data)
X_train = data_processor.transform(train_data)
X_test = data_processor.transform(test_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



77090 энграмм в словаре.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


**67284** энграммы без учета целых слов

**230656** энграмм с целыми словами 

**71877** энграмм с частотными словами




In [ ]:
# обучаем модель
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

cls = MultinomialNB()
# cls = LogisticRegression(max_iter=100)
cls.fit(X_train, train_labels)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Вычисляем ответы на тестовой выборке и сравниваем ответы для первых 20 текстов



In [ ]:
pred_labels = cls.predict(X_test)
print(test_labels[:20])
print(pred_labels[:20])

['sme', 'myv', 'sme', 'myv', 'mrj', 'sme', 'sme', 'udm', 'udm', 'sme', 'mdf', 'mrj', 'sme', 'olo', 'olo', 'mhr', 'koi', 'vep', 'myv', 'sme']
['sme' 'myv' 'sme' 'myv' 'mrj' 'sme' 'sme' 'udm' 'udm' 'sme' 'mdf' 'mrj'
 'sme' 'olo' 'olo' 'mhr' 'koi' 'vep' 'myv' 'sme']


## Измерение качества

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

accuracy = accuracy_score(test_labels, pred_labels)
print("Корректность: {:.2f}".format(100 * accuracy))
prec, rec, f1, sup = precision_recall_fscore_support(test_labels, pred_labels)
print("Точность:")
for label, x in zip(cls.classes_, prec):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")
print("Полнота:")
for label, x in zip(cls.classes_, rec):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")
print("F1-мера:")
for label, x in zip(cls.classes_, f1):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")

Корректность: 96.48
Точность:
fit: 67.68	fkv: 77.00	izh: 0.00	kca: 99.60	koi: 78.10	kpv: 89.02	krl: 90.50	liv: 99.37	lud: 97.22	mdf: 93.13	mhr: 97.35	mns: 99.11	mrj: 94.85	myv: 94.86	nio: 100.00	olo: 88.70	sjd: 96.67	sjk: 0.00	sju: 100.00	sma: 94.38	sme: 99.73	smj: 85.48	smn: 96.75	sms: 98.12	udm: 98.45	vep: 90.31	vot: 0.00	vro: 98.64	yrk: 100.00	
Полнота:
fit: 97.08	fkv: 30.43	izh: 0.00	kca: 99.60	koi: 68.87	kpv: 92.25	krl: 80.86	liv: 89.77	lud: 72.54	mdf: 97.03	mhr: 97.75	mns: 98.67	mrj: 91.31	myv: 97.06	nio: 100.00	olo: 97.78	sjd: 87.88	sjk: 0.00	sju: 16.13	sma: 99.14	sme: 97.38	smj: 97.25	smn: 99.38	sms: 98.87	udm: 97.17	vep: 96.40	vot: 0.00	vro: 99.05	yrk: 94.59	
F1-мера:
fit: 79.76	fkv: 43.62	izh: 0.00	kca: 99.60	koi: 73.20	kpv: 90.61	krl: 85.41	liv: 94.33	lud: 83.09	mdf: 95.04	mhr: 97.55	mns: 98.89	mrj: 93.04	myv: 95.95	nio: 100.00	olo: 93.02	sjd: 92.06	sjk: 0.00	sju: 27.78	sma: 96.70	sme: 98.55	smj: 90.98	smn: 98.05	sms: 98.50	udm: 97.81	vep: 93.26	vot: 0.00	vro: 98.85	yrk: 97.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


baseline - Корректность: 96.52

whole_words - Корректность: 97.03

sub_words - 96.48

 Считаем матрицу ошибок



In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(test_labels, pred_labels)
print(" "*6, end="")
for label in cls.classes_:
    print("{:<6}".format(label), end="")
print("")
for i, label in enumerate(cls.classes_):
    print("{:<6}".format(label), end="")
    for x in conf_matrix[i]:
        print("{:<6}".format(x), end="")
    print("")

      fit   fkv   izh   kca   koi   kpv   krl   liv   lud   mdf   mhr   mns   mrj   myv   nio   olo   sjd   sjk   sju   sma   sme   smj   smn   sms   udm   vep   vot   vro   yrk   
fit   1263  13    0     0     0     0     1     0     0     0     0     0     0     0     0     8     0     0     0     0     5     0     0     0     0     4     0     7     0     
fkv   358   164   0     0     0     0     1     0     0     0     0     0     0     0     0     7     0     0     0     1     2     1     0     0     0     0     0     5     0     
izh   17    0     0     0     0     0     1     0     0     0     0     0     0     0     0     1     0     0     0     0     0     0     0     0     0     0     0     1     0     
kca   0     0     0     250   0     0     0     0     0     1     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     
koi   0     0     0     1     1405  562   0     0     0     9     19    0     3     15    0    

## Вычисление значимости признаков

### Значимость признаков в уже обученной модели

In [ ]:
import numpy as np

def get_feature_importance(cls):
    # cls.coef_.shape = (n_classes, n_features)
    # индексы максимальных классов для каждого признака
    best_classes_for_features = np.argmax(cls.coef_, axis=0) # n_features
    # максимальные веса для каждого признака
    best_weights_for_features = np.max(cls.coef_, axis=0)
    # средние веса для каждого признака
    mean_weights_for_features = np.mean(cls.coef_, axis=0)
    # разность между максимумом и средним
    feature_importance = best_weights_for_features - mean_weights_for_features
    # для каждого класса запоминаем признаки
    answer = defaultdict(dict)
    for i, (label, importance) in enumerate(zip(best_classes_for_features, feature_importance)):
        label = cls.classes_[label]
        answer[label][i] = importance
    return answer

In [ ]:
from sklearn.preprocessing import label_binarize # преобразование категориальных значений в 0-1 вектора/матрицы
from sklearn.utils.extmath import safe_sparse_dot # перемножение разреженных матриц

# преобразуем классы в 0-1 матрицу 
# (ставим sparse_output=True, чтобы потом не перемножать разреженную матрицу на обычную)
Y_train = label_binarize(train_labels, cls.classes_, sparse_output=True)
# матрица размера число признаков на число классов
feature_counts_by_classes = safe_sparse_dot(Y_train.T, (X_train >= 1), dense_output=True)

In [ ]:
importances_by_classes = get_feature_importance(cls)
for k, label in enumerate(cls.classes_):
    print(label)
    for feat, importance in sorted(importances_by_classes[label].items(), key=lambda x: -x[1])[:10]:
        feat_string = data_processor.ngrams[feat]
        print(feat_string, "{:.2f}".format(importance), end=" " * (10-len(feat_string)))
        for other_label, count in zip(cls.classes_, feature_counts_by_classes[:, feat]):
            print("{}:{}".format(other_label, count), end=" ")
        print("")
    print("")

fit
ttä 5.43       fit:755 fkv:138 izh:2 kca:0 koi:0 kpv:0 krl:128 liv:0 lud:0 mdf:0 mhr:0 mns:0 mrj:0 myv:0 nio:0 olo:396 sjd:0 sjk:0 sju:0 sma:0 sme:26 smj:0 smn:9 sms:36 udm:0 vep:4 vot:0 vro:177 yrk:0 
tä 5.32        fit:1927 fkv:654 izh:13 kca:0 koi:2 kpv:0 krl:742 liv:27 lud:2 mdf:0 mhr:0 mns:0 mrj:0 myv:0 nio:0 olo:1598 sjd:0 sjk:2 sju:0 sma:22 sme:108 smj:3 smn:597 sms:712 udm:0 vep:311 vot:4 vro:7852 yrk:0 
hee 5.21       fit:530 fkv:202 izh:0 kca:0 koi:0 kpv:0 krl:1 liv:0 lud:0 mdf:0 mhr:1 mns:0 mrj:1 myv:0 nio:0 olo:1 sjd:0 sjk:0 sju:0 sma:122 sme:88 smj:0 smn:134 sms:9 udm:0 vep:0 vot:0 vro:21 yrk:0 
hää 5.17       fit:431 fkv:151 izh:1 kca:0 koi:0 kpv:0 krl:0 liv:0 lud:0 mdf:0 mhr:0 mns:0 mrj:0 myv:0 nio:0 olo:1 sjd:0 sjk:0 sju:0 sma:0 sme:4 smj:0 smn:69 sms:45 udm:0 vep:0 vot:0 vro:686 yrk:0 
tää 5.14       fit:481 fkv:113 izh:0 kca:0 koi:0 kpv:0 krl:0 liv:0 lud:0 mdf:0 mhr:0 mns:0 mrj:0 myv:0 nio:0 olo:3 sjd:0 sjk:0 sju:0 sma:0 sme:11 smj:0 smn:302 sms:316 udm:0 vep:0 vo

### Отбор признаков

In [ ]:
import numpy as np
from sklearn.utils.extmath import safe_sparse_dot

def get_count_feature_importance(X, Y, classes):
    """
    X --- (возможно, разреженная) матрица тексты-признаки, X.shape = (m, n),
    Y --- матрица m \times k, содержащий метки классов, закодированные в 0/1-формате
    """
    feature_counts_by_classes = safe_sparse_dot(Y.T, (X >= 1), dense_output=True)
    # log_feature_count[i] = log_2(1+n_i) = log_2(1 + \sum_k n_{ki})
    log_feature_count = np.log2(1.0 + feature_counts_by_classes.sum(axis=0))
    feature_probs_by_classes = feature_counts_by_classes / feature_counts_by_classes.sum(axis=0)
    K = Y.shape[1] # число классов
    feature_weights_by_classes = log_feature_count * (feature_probs_by_classes - 1/K)
    answer = defaultdict(dict)
    for i, feat_weights in enumerate(feature_weights_by_classes.T):
        for label, weight in zip(classes, feat_weights):
            if weight > 0:
                answer[label][i] = weight
    return answer

In [ ]:
from sklearn.preprocessing import label_binarize

Y = label_binarize(train_labels, classes=cls.classes_)
importances_by_classes = get_count_feature_importance(X_train, Y, classes=cls.classes_)
for k, label in enumerate(cls.classes_):
    print(label)
    for feat, importance in sorted(importances_by_classes[label].items(), key=lambda x: -x[1])[:20]:
        feat_string = data_processor.ngrams[feat]
        print(feat_string, "{:.2f}".format(importance), end=" " * (10-len(feat_string)))
        for other_label, count in zip(cls.classes_, feature_counts_by_classes[:, feat]):
            print("{}:{}".format(other_label, count), end=" ")
        print("")
    print("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


fit
kha 5.41       fit:311 fkv:74 izh:0 kca:0 koi:1 kpv:0 krl:0 liv:0 lud:1 mdf:1 mhr:0 mns:0 mrj:1 myv:0 nio:0 olo:0 sjd:0 sjk:0 sju:0 sma:1 sme:37 smj:0 smn:0 sms:1 udm:0 vep:26 vot:0 vro:32 yrk:0 
uue 5.35       fit:102 fkv:15 izh:0 kca:0 koi:0 kpv:0 krl:0 liv:0 lud:0 mdf:0 mhr:0 mns:0 mrj:0 myv:0 nio:0 olo:0 sjd:0 sjk:0 sju:0 sma:0 sme:0 smj:0 smn:0 sms:0 udm:0 vep:1 vot:0 vro:10 yrk:0 
..! 5.32       fit:78 fkv:4 izh:0 kca:0 koi:0 kpv:0 krl:0 liv:0 lud:0 mdf:0 mhr:1 mns:0 mrj:0 myv:0 nio:0 olo:0 sjd:0 sjk:0 sju:0 sma:0 sme:9 smj:0 smn:0 sms:0 udm:0 vep:0 vot:0 vro:0 yrk:0 
thä 5.31       fit:190 fkv:68 izh:0 kca:0 koi:0 kpv:0 krl:0 liv:0 lud:0 mdf:0 mhr:0 mns:0 mrj:0 myv:0 nio:0 olo:17 sjd:0 sjk:0 sju:0 sma:0 sme:0 smj:0 smn:0 sms:0 udm:0 vep:0 vot:0 vro:1 yrk:0 
eän 5.21       fit:295 fkv:37 izh:0 kca:0 koi:0 kpv:0 krl:18 liv:0 lud:0 mdf:0 mhr:0 mns:0 mrj:0 myv:0 nio:0 olo:16 sjd:0 sjk:0 sju:1 sma:15 sme:40 smj:14 smn:0 sms:36 udm:0 vep:0 vot:0 vro:4 yrk:0 
ko' 5.06       fit:64 

### Фильтрация признаков

Оставим в каждом классе только по 400 самых полезных признаков

In [ ]:
Y_train = label_binarize(train_labels, classes=cls.classes_)
# importances_by_classes = {
#    "cz": {"a": 0.1, "b": 0.12, ...}, "sk": {"acd": 2.34, ...}, ...
# }
importances_by_classes = get_count_feature_importance(X_train, Y_train, classes=cls.classes_)
useful_features = set()
for label in cls.classes_:
    for feat, importance in sorted(importances_by_classes[label].items(), key=lambda x: -x[1])[:400]:
        useful_features.add(feat)
useful_features = sorted(useful_features)
X_train_new = X_train[:,useful_features]
X_test_new = X_test[:,useful_features]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


Обучаем и тестируем классификатор

In [ ]:
from sklearn.metrics import f1_score

cls_small = MultinomialNB().fit(X_train_new, train_labels)
#cls_small = LogisticRegression().fit(X_train_new, train_labels)
pred_labels = cls_small.predict(X_test_new)

accuracy = accuracy_score(test_labels, pred_labels)
print("Корректность: {:.2f}".format(100 * accuracy))
f1 = f1_score(test_labels, pred_labels, average=None)
for label, x in zip(cls_small.classes_, f1):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")

Корректность: 95.73
fit: 79.33	fkv: 61.15	izh: 0.00	kca: 99.60	koi: 71.53	kpv: 88.01	krl: 83.40	liv: 95.24	lud: 88.43	mdf: 95.09	mhr: 96.99	mns: 98.03	mrj: 92.81	myv: 94.79	nio: 96.77	olo: 91.48	sjd: 95.52	sjk: 0.00	sju: 60.00	sma: 96.37	sme: 97.85	smj: 89.84	smn: 97.75	sms: 97.94	udm: 96.37	vep: 93.15	vot: 0.00	vro: 97.70	yrk: 98.65	


Тестируем для разного количества признаков

In [ ]:
importances_by_classes = get_count_feature_importance(X_train, Y_train, classes=cls.classes_)
for n_feat_for_class in [100, 200, 400, 1000, 2000]:
    useful_features = set()
    for label in cls.classes_:
        for feat, importance in sorted(importances_by_classes[label].items(), key=lambda x: -x[1])[:n_feat_for_class]:
            useful_features.add(feat)
    useful_features = sorted(useful_features)
    X_train_new = X_train[:,useful_features]
    X_test_new = X_test[:,useful_features]
    # cls_small = LogisticRegression().fit(X_train_new, train_labels)
    cls_small = MultinomialNB().fit(X_train_new, train_labels)
    pred_labels = cls_small.predict(X_test_new)

    accuracy = accuracy_score(test_labels, pred_labels)
    print("{} признаков, корректность: {:.2f}".format(len(useful_features), 100 * accuracy))
    f1 = f1_score(test_labels, pred_labels, average=None)
    for label, x in zip(cls_small.classes_, f1):
        print("{}: {:.2f}".format(label, 100*x), end="\t")
    print("")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


2678 признаков, корректность: 92.45
fit: 75.10	fkv: 53.71	izh: 9.52	kca: 97.98	koi: 65.57	kpv: 84.10	krl: 80.30	liv: 91.52	lud: 88.11	mdf: 89.17	mhr: 95.63	mns: 96.60	mrj: 91.48	myv: 91.21	nio: 96.77	olo: 86.88	sjd: 89.86	sjk: 66.67	sju: 44.44	sma: 95.30	sme: 93.45	smj: 86.88	smn: 97.02	sms: 96.40	udm: 92.66	vep: 87.10	vot: 33.33	vro: 94.43	yrk: 98.20	
5228 признаков, корректность: 94.38
fit: 77.97	fkv: 56.91	izh: 9.52	kca: 99.00	koi: 69.47	kpv: 86.64	krl: 81.36	liv: 93.09	lud: 88.77	mdf: 92.63	mhr: 96.44	mns: 97.76	mrj: 92.09	myv: 93.37	nio: 96.77	olo: 89.50	sjd: 91.18	sjk: 33.33	sju: 45.57	sma: 95.89	sme: 96.02	smj: 87.49	smn: 97.53	sms: 97.33	udm: 95.02	vep: 90.57	vot: 0.00	vro: 96.47	yrk: 96.86	
9993 признаков, корректность: 95.73
fit: 79.33	fkv: 61.15	izh: 0.00	kca: 99.60	koi: 71.53	kpv: 88.01	krl: 83.40	liv: 95.24	lud: 88.43	mdf: 95.09	mhr: 96.99	mns: 98.03	mrj: 92.81	myv: 94.79	nio: 96.77	olo: 91.48	sjd: 95.52	sjk: 0.00	sju: 60.00	sma: 96.37	sme: 97.85	smj: 89.84	smn: 97.75	sms:

## Для baseline

2675 признаков, корректность: 92.26

5225 признаков, корректность: 94.22

10000 признаков, корректность: 95.61

21616 признаков, корректность: 96.30

35365 признаков, корректность: 96.50

## Для whole word

2738 признаков, корректность: 92.53

5331 признаков, корректность: 94.41

10396 признаков, корректность: 95.84

23855 признаков, корректность: 96.55

42810 признаков, корректность: 96.91

## Для sub word

2678 признаков, корректность: 92.45

5228 признаков, корректность: 94.38

9993 признаков, корректность: 95.73

21601 признаков, корректность: 96.30

35414 признаков, корректность: 96.48





### Взвешивание признаков

In [ ]:
from scipy.sparse import csr_matrix

class SparseWeightedDataProcessor(SparseDataProcessor):

    def weight_features(self, X, Y):
        raise NotImplementedError("You should implement feature weighting in your derived class.")

    def fit(self, data, labels):
        super().fit(data)
        X = super().transform(data)
        classes = sorted(set(labels))
        Y = label_binarize(labels, classes)
        self.feature_weights_ = self.weight_features(X, Y)
        return self

    def transform(self, data):
        answer = super().transform(data)
        answer = answer.multiply(self.feature_weights_)
        return answer

In [ ]:
class TfIdfSparseDataProcessor(SparseWeightedDataProcessor):

    def weight_features(self, X, Y=None):
        feature_counts = np.array((X >= 1).astype("int").sum(axis=0))
        return 1.0 / np.log2(1.0 + feature_counts)

In [ ]:
# преобразуем данные
data_processor = TfIdfSparseDataProcessor(min_count=3, ngram_length=3, whole_word = True, sub_word = True)
data_processor.fit(train_data, train_labels)
X_train_new = data_processor.transform(train_data)
X_test_new = data_processor.transform(test_data)
# обучаем модель
#cls = MultinomialNB()
cls_small = MultinomialNB().fit(X_train_new, train_labels)
# cls_small = LogisticRegression().fit(X_train_new, train_labels)
pred_labels = cls_small.predict(X_test_new)

accuracy = accuracy_score(test_labels, pred_labels)
print("Корректность: {:.2f}".format(100 * accuracy))
f1 = f1_score(test_labels, pred_labels, average=None)
for label, x in zip(cls_small.classes_, f1):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



77090 энграмм в словаре.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



Корректность: 93.38
fit: 34.69	fkv: 0.00	izh: 0.00	kca: 92.96	koi: 40.75	kpv: 84.78	krl: 22.65	liv: 76.92	lud: 20.47	mdf: 94.91	mhr: 94.86	mns: 84.69	mrj: 88.74	myv: 95.48	nio: 96.55	olo: 81.10	sjd: 65.31	sjk: 0.00	sju: 0.00	sma: 96.55	sme: 97.29	smj: 76.92	smn: 97.27	sms: 97.91	udm: 96.19	vep: 87.55	vot: 0.00	vro: 93.26	yrk: 58.60	


baseline - Корректность: 94.98

whole_word - 92.82

sub_word - 93.38

**Tf-idf**-векторизатор начинает занижать вес признаков, начиная с частоты 1. Для большей гибкости можно установить порог, до которого веса признаков равны 1, а после которого начинают уменьшаться.

In [ ]:
class ThresholdTfIdfSparseDataProcessor(SparseWeightedDataProcessor):

    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold

    def weight_features(self, X, Y=None):
        feature_counts = (X >= 1).astype("float").sum(axis=0)[0]
        relative_feature_counts = np.maximum(feature_counts / self.threshold, 1.0)
        return 1.0 / np.log2(1.0 + relative_feature_counts)

In [ ]:
# преобразуем данные
data_processor = ThresholdTfIdfSparseDataProcessor(min_count=3, ngram_length=3, threshold=10.0, whole_word = True, sub_word = True)
data_processor.fit(train_data, train_labels)
X_train_new = data_processor.transform(train_data)
X_test_new = data_processor.transform(test_data)
# обучаем модель
cls_small = MultinomialNB().fit(X_train_new, train_labels)
# cls_small = LogisticRegression().fit(X_train_new, train_labels)
pred_labels = cls_small.predict(X_test_new)

accuracy = accuracy_score(test_labels, pred_labels)
print("Корректность: {:.2f}".format(100 * accuracy))
f1 = f1_score(test_labels, pred_labels, average=None)
for label, x in zip(cls_small.classes_, f1):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



77090 энграмм в словаре.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



Корректность: 95.39
fit: 68.14	fkv: 6.75	izh: 0.00	kca: 99.20	koi: 64.58	kpv: 88.84	krl: 61.13	liv: 92.73	lud: 61.21	mdf: 95.98	mhr: 96.35	mns: 97.52	mrj: 91.37	myv: 96.30	nio: 100.00	olo: 88.58	sjd: 92.06	sjk: 0.00	sju: 32.43	sma: 96.98	sme: 98.16	smj: 86.00	smn: 97.78	sms: 98.36	udm: 97.20	vep: 92.11	vot: 0.00	vro: 96.16	yrk: 94.79	


baseline - Корректность: 97.06

whole_word - Корректность: 96.70

sub_word - Корректность: 95.39


В библиотеке `sklearn` используется немного другая реализация `tf-idf`

In [ ]:
class TfIdfSklearnDataProcessor(SparseWeightedDataProcessor):

    def weight_features(self, X, Y=None):
        feature_counts = (X >= 1).astype("int").sum(axis=0)[0]
        N = X.shape[0] # число объектов
        return 1.0 + np.log2(N / feature_counts)

In [ ]:
# преобразуем данные
data_processor = TfIdfSklearnDataProcessor(min_count=3, ngram_length=3, whole_word = True)
data_processor.fit(train_data, train_labels)
X_train_new = data_processor.transform(train_data)
X_test_new = data_processor.transform(test_data)
# обучаем модель

cls_small = MultinomialNB().fit(X_train_new, train_labels)
# cls_small = LogisticRegression().fit(X_train_new, train_labels)
pred_labels = cls_small.predict(X_test_new)

accuracy = accuracy_score(test_labels, pred_labels)
print("Корректность: {:.2f}".format(100 * accuracy))
f1 = f1_score(test_labels, pred_labels, average=None)
for label, x in zip(cls_small.classes_, f1):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



230656 энграмм в словаре.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



Корректность: 96.88
fit: 82.75	fkv: 66.08	izh: 0.00	kca: 99.60	koi: 75.44	kpv: 91.23	krl: 84.79	liv: 96.51	lud: 92.67	mdf: 96.04	mhr: 97.91	mns: 98.90	mrj: 94.11	myv: 96.54	nio: 100.00	olo: 94.23	sjd: 95.52	sjk: 0.00	sju: 60.38	sma: 97.06	sme: 98.61	smj: 90.56	smn: 98.02	sms: 98.50	udm: 98.18	vep: 95.14	vot: 0.00	vro: 98.85	yrk: 99.11	


baseline - Корректность: 96.81

whole_word - 97.90

sub_word - 96.88






In [ ]:
from scipy.stats import entropy

class TfIdfClassDataProcessor(SparseWeightedDataProcessor):

    def weight_features(self, X, Y=None):
        feature_counts_by_classes = safe_sparse_dot(Y.T, (X >= 1), dense_output=True)
        feature_counts = feature_counts_by_classes.sum(axis=0)
        feature_probs_by_classes = feature_counts_by_classes / feature_counts
        probs_entropy = entropy(feature_probs_by_classes, base=2.0, axis=0)
        probs_entropy = 1.0 + probs_entropy * np.log2(feature_counts)
        weights = 1.0 / probs_entropy
        return weights

In [ ]:
# преобразуем данные
data_processor = TfIdfClassDataProcessor(min_count=3, ngram_length=3, whole_word = True, sub_word = True)
data_processor.fit(train_data, train_labels)
X_train_new = data_processor.transform(train_data)
X_test_new = data_processor.transform(test_data)
# обучаем модель
cls_small = MultinomialNB().fit(X_train_new, train_labels)
# cls_small = LogisticRegression().fit(X_train_new, train_labels)
pred_labels = cls_small.predict(X_test_new)

accuracy = accuracy_score(test_labels, pred_labels)
print("Корректность: {:.2f}".format(100 * accuracy))
f1 = f1_score(test_labels, pred_labels, average=None)
for label, x in zip(cls_small.classes_, f1):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



77090 энграмм в словаре.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log2
  # Remove the CWD from sys.path while we load stuff.



Корректность: 93.85
fit: 42.00	fkv: 8.17	izh: 0.00	kca: 99.60	koi: 54.50	kpv: 86.40	krl: 57.70	liv: 93.01	lud: 67.58	mdf: 92.10	mhr: 94.81	mns: 97.04	mrj: 92.57	myv: 93.30	nio: 100.00	olo: 81.64	sjd: 92.06	sjk: 0.00	sju: 32.43	sma: 96.37	sme: 96.77	smj: 82.77	smn: 97.64	sms: 98.21	udm: 96.29	vep: 88.44	vot: 0.00	vro: 95.59	yrk: 96.26	


baseline - Корректность: 94.00

whole_word - 96.92

sub_word - 93.85